# BYOC (Bring Your Own Container) 를 이용한 Amazon SageMaker Multi-Model Endpoints 구축
[Amazon SageMaker multi-model endpoints](https://docs.aws.amazon.com/sagemaker/latest/dg/multi-model-endpoints.html)를 이용해서, 고객들은 수천개의 모델에 대한 endpoint 을 생성할 수 있습니다. 

이러한 endpoint 는 공통된 추론 컨테이너에서 제공할 수 있는 많은 모델 중 하나가 요청시 호출되어야 하고 드물게 호출되는 모델이 추가 대기 시간을 발생시키는 것이 허용되는 사용 사례에 매우 적합합니다. 

지속적으로 낮은 추론 지연 시간이 필요한 애플리케이션의 경우 전통적인  엔드포인트가 여전히 최선의 선택입니다.

Amazon SageMaker는 필요에 따라 다중 모델 엔드포인트에 대한 모델 로드 및 언로드를 관리합니다. 

특정 모델에 대한 호출 요청이 이루어지면 Amazon SageMaker는 해당 모델에 할당된 인스턴스로 요청을 라우팅하고 S3에서 해당 인스턴스로 모델 아티팩트를 다운로드하고 컨테이너의 메모리로 모델 로드를 시작합니다. 

로드가 완료되는 즉시 Amazon SageMaker는 요청된 호출을 수행하고 결과를 반환합니다. 모델이 선택한 인스턴스의 메모리에 이미 로드된 경우 다운로드 및 로드 단계를 건너뛰고 호출이 즉시 수행됩니다.

다중 모델 엔드포인트에서 여러 모델을 제공하는 추론 컨테이너의 경우 특정 모델을 load, list, get, unload 및 invoke 하기 위해 [추가적인 API](https://docs.aws.amazon.com/sagemaker/latest/dg/build-multi-model-build-container.html)를 구현해야 합니다.
이 노트북은 이러한 API를 구현하는 추론 컨테이너를 구축하는 방법을 보여줍니다.

**Note**: 이 노트북은 Docker 컨테이너를 구축하기 때문에 Amazon SageMaker Studio에서는 실행되지 않습니다.

이 노트북은 Amazon SageMaker 노트북 인스턴스에서 SageMaker Python SDK 버전 2.15.3을 실행하는 `conda_mxnet_p36` 커널로 테스트되었습니다.

---

### 목차

1. [Multi Model Server (MMS) 소개](#Introduction-to-Multi-Model-Server-(MMS))
  1. [Out Of Memory 상태 처리](#Handling-Out-Of-Memory-conditions)
  2. [SageMaker 추론 Toolkit](#SageMaker-Inference-Toolkit)
2. [MMS를 사용하여 컨테이너 구축 및 등록](#Building-and-registering-a-container-using-MMS)
3. [환경설정](#Set-up-the-environment)
4. [모델 artifact의 S3 업로드](#Upload-model-artifacts-to-S3)
5. [다중모델 endpoint 생성](#Create-a-multi-model-endpoint)
  3. [호스팅안으로 모델 입수](#Import-models-into-hosting)
  4. [endpoint configuration 생성](#Create-endpoint-configuration)
  5. [endpoint 생성](#Create-endpoint)
6. [models 호출](#Invoke-models)
  6. [모델을 endpoint에 추가](#Add-models-to-the-endpoint)
  7. [모델 갱신](#Updating-a-model)
7. [(선택) 호스팅 자원 삭제](#(Optional)-Delete-the-hosting-resources)

## Multi Model Server (MMS) 소개

[Multi Model Server](https://github.com/awslabs/multi-model-server)는 기계 학습 모델을 서비스하기 위한 오픈 소스 프레임워크입니다.

단일 컨테이너 내에서 여러 모델을 호스팅하고, 모델을 컨테이너에 동적으로 로드 및 언로드하고, 지정된 로드된 모델에 대한 추론을 수행하기 위해 다중 모델 엔드포인트에 필요한 HTTP 프론트엔드 및 모델 관리 기능을 제공합니다.

MMS는 사용자의 알고리즘을 구현할 수 있는 플러그형 사용자 정의 백엔드 핸들러를 지원합니다. 이 예제에서는 MXNet 모델에 대한 로드 및 추론을 지원하는 핸들러를 사용합니다. 이에 대해서는 아래에서 살펴보겠습니다.

In [ ]:
!cat container/model_handler.py


주목할만한 것은 `handle(data, context)` 및 `initialize(self, context)` 메소드입니다.

`initialize` 메소드는 모델이 메모리에 로드될 때 호출됩니다. 이 예에서는 `model_dir`의 모델 아티팩트를 MXNet으로 로드합니다.

`handle` 메서드는 모델을 호출할 때 호출됩니다. 이 예에서는 입력 페이로드의 유효성을 검사한 다음 입력을 MXNet으로 전달하여 출력을 반환합니다.

이 핸들러 클래스는 컨테이너에 로드되는 모든 모델에 대해 인스턴스화되므로 핸들러의 상태는 모델 간에 공유되지 않습니다.

### Out Of Memory 상태 처리

MXNet이 메모리 부족으로 인해 모델을 로드하지 못하면 `MemoryError`가 발생합니다. 메모리 부족이나 기타 리소스 제약으로 인해 모델을 로드할 수 없을 때마다 `MemoryError`가 발생해야 합니다. 

MMS는 `MemoryError`를 해석하고 SageMaker에 507 HTTP 상태 코드를 반환합니다. 여기서 SageMaker는 요청된 모델을 로드할 수 있도록 리소스를 회수하기 위해 사용하지 않는 모델에 대해 언로드를 시작합니다.

### SageMaker 추론 Toolkit

MMS 는 frontend 서버를 위해 [다양한 설정](https://github.com/awslabs/multi-model-server/blob/master/docker/advanced_settings.md#description-of-config-file-settings)을 지원합니다. 

[SageMaker 추론 Toolkit](https://github.com/aws/sagemaker-inference-toolkit)는 SageMaker 다중 모델 엔드포인트와 호환되는 방식으로 MMS를 부트스트랩하면서 모델당 작업자 수와 같은 중요한 성능 매개변수를 조정할 수 있는 라이브러리입니다. 

이 예제의 추론 컨테이너는 __`container/dockerd-entrypoint.py`__ 파일에서 보는 것처럼 MMS를 시작하기 위해 추론 Toolkit을 사용합니다.

## MMS를 사용하여 컨테이너 구축 및 등록

아래 셸 스크립트는 MMS를 프런트 엔드로 사용하는 Docker 이미지(SageMaker 추론 Toolkit을 통해 구성)와 위에서 본 `container/model_handler.py`를 백엔드 핸들러로 빌드합니다. 

그런 다음 이미지를 계정의 ECR 저장소에 업로드합니다.

In [ ]:
%%sh

# The name of our algorithm
algorithm_name=demo-sagemaker-multimodel

cd container

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-ap-northeast-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build -q -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

## 환경설정

다중 모델 엔드포인트에서 호출할 수 있는 모델 아티팩트가 위치할 S3 버킷 및 prefix를 정의합니다.

또한 위에서 생성한 모델 아티팩트 및 ECR 이미지에 대한 액세스 권한을 SageMaker에 부여하는 IAM 역할을 정의합니다.

In [ ]:
!pip install -qU awscli boto3 sagemaker

In [ ]:
import boto3
from sagemaker import get_execution_role

sm_client = boto3.client(service_name="sagemaker")
runtime_sm_client = boto3.client(service_name="sagemaker-runtime")

account_id = boto3.client("sts").get_caller_identity()["Account"]
region = boto3.Session().region_name

bucket = "sagemaker-{}-{}".format(region, account_id)
prefix = "demo-multimodel-endpoint"

role = get_execution_role()

## 모델 artifact를 S3에 업로드

이 예에서는 ImageNet 데이터셋에서 사전 훈련된 ResNet 18 및 ResNet 152 모델을 사용합니다. 
먼저 MXNet의 모델 zoo에서 모델을 다운로드한 다음 S3에 업로드합니다.

In [ ]:
import mxnet as mx
import os
import tarfile

model_path = "http://data.mxnet.io/models/imagenet/"

mx.test_utils.download(
    model_path + "resnet/18-layers/resnet-18-0000.params", None, "data/resnet_18"
)
mx.test_utils.download(
    model_path + "resnet/18-layers/resnet-18-symbol.json", None, "data/resnet_18"
)
mx.test_utils.download(model_path + "synset.txt", None, "data/resnet_18")

with open("data/resnet_18/resnet-18-shapes.json", "w") as file:
    file.write('[{"shape": [1, 3, 224, 224], "name": "data"}]')

with tarfile.open("data/resnet_18.tar.gz", "w:gz") as tar:
    tar.add("data/resnet_18", arcname=".")

In [ ]:
mx.test_utils.download(
    model_path + "resnet/152-layers/resnet-152-0000.params", None, "data/resnet_152"
)
mx.test_utils.download(
    model_path + "resnet/152-layers/resnet-152-symbol.json", None, "data/resnet_152"
)
mx.test_utils.download(model_path + "synset.txt", None, "data/resnet_152")

with open("data/resnet_152/resnet-152-shapes.json", "w") as file:
    file.write('[{"shape": [1, 3, 224, 224], "name": "data"}]')

with tarfile.open("data/resnet_152.tar.gz", "w:gz") as tar:
    tar.add("data/resnet_152", arcname=".")

In [ ]:
from botocore.client import ClientError
import os

s3 = boto3.resource("s3")
try:
    s3.meta.client.head_bucket(Bucket=bucket)
except ClientError:
    s3.create_bucket(Bucket=bucket, CreateBucketConfiguration={"LocationConstraint": region})

models = {"resnet_18.tar.gz", "resnet_152.tar.gz"}

for model in models:
    key = os.path.join(prefix, model)
    with open("data/" + model, "rb") as file_obj:
        s3.Bucket(bucket).Object(key).upload_fileobj(file_obj)

## 다중 모델 endpoint 생성
### 호스팅안으로 모델 입수

다중 모델 endpoint 에 대한 모델 엔터티를 만들 때 컨테이너의 `ModelDataUrl`은 endpoint 에서 호출할 수 있는 모델 아티팩트가 있는 S3 prefix 입니다. 

나머지 S3 경로는 모델을 호출할 때 지정됩니다.

컨테이너의 `Mode`는 컨테이너가 여러 모델을 호스트할 것임을 나타내기 위해 `MultiModel` 로 지정됩니다.

In [ ]:
from time import gmtime, strftime

model_name = "DEMO-MultiModelModel" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model_url = "https://s3-{}.amazonaws.com/{}/{}/".format(region, bucket, prefix)
container = "{}.dkr.ecr.{}.amazonaws.com/{}:latest".format(
    account_id, region, "demo-sagemaker-multimodel"
)

print("Model name: " + model_name)
print("Model data Url: " + model_url)
print("Container image: " + container)

container = {"Image": container, "ModelDataUrl": model_url, "Mode": "MultiModel"}

create_model_response = sm_client.create_model(
    ModelName=model_name, ExecutionRoleArn=role, Containers=[container]
)

print("Model Arn: " + create_model_response["ModelArn"])

### endpoint configuration 생성

endpoint configuration 단일 모델 엔드포인트와 동일한 방식으로 작동합니다.

In [ ]:
endpoint_config_name = "DEMO-MultiModelEndpointConfig-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("Endpoint config name: " + endpoint_config_name)

create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "InstanceType": "ml.m5.xlarge",
            "InitialInstanceCount": 2,
            "InitialVariantWeight": 1,
            "ModelName": model_name,
            "VariantName": "AllTraffic",
        }
    ],
)

print("Endpoint config Arn: " + create_endpoint_config_response["EndpointConfigArn"])

### endpoint 생성

마찬가지로 endpoint 생성은 단일 모델 endpoint 와 동일한 방식으로 작동합니다.

In [ ]:
import time

endpoint_name = "DEMO-MultiModelEndpoint-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("Endpoint name: " + endpoint_name)

create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name, EndpointConfigName=endpoint_config_name
)
print("Endpoint Arn: " + create_endpoint_response["EndpointArn"])

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Endpoint Status: " + status)

print("Waiting for {} endpoint to be in service...".format(endpoint_name))
waiter = sm_client.get_waiter("endpoint_in_service")
waiter.wait(EndpointName=endpoint_name)

## model 호출

이제 이전에 S3에 업로드한 모델을 호출합니다. SageMaker는 첫번째 호출시 S3에서 인스턴스로 모델 아티팩트를 다운로드하고 컨테이너에 로드하기 때문에 조금 느릴 수 있습니다.

먼저 고양이 이미지를 페이로드로 다운로드하여 모델을 호출한 다음 InvokeEndpoint를 호출하여 ResNet 18 모델을 호출합니다. 

`TargetModel` 필드는 모델 생성 시 `ModelDataUrl`에 지정된 S3 prefix 와 연결되어 S3에서 모델의 위치를 생성합니다.

In [ ]:
fname = mx.test_utils.download(
    "https://github.com/dmlc/web-data/blob/master/mxnet/doc/tutorials/python/predict_image/cat.jpg?raw=true",
    "cat.jpg",
)

with open(fname, "rb") as f:
    payload = f.read()

In [ ]:
%%time

import json

response = runtime_sm_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/x-image",
    TargetModel="resnet_18.tar.gz",  # this is the rest of the S3 path where the model artifacts are located
    Body=payload,
)

print(*json.loads(response["Body"].read()), sep="\n")


동일한 ResNet 18 모델을 두 번째로 호출하면 이미 인스턴스에 다운로드되어 컨테이너에 로드되므로 추론이 더 빠릅니다.

In [ ]:
%%time

response = runtime_sm_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/x-image",
    TargetModel="resnet_18.tar.gz",
    Body=payload,
)

print(*json.loads(response["Body"].read()), sep="\n")

### 다른 모델 호출

다중 모델 엔드포인트의 강력함을 이용하여 다른 모델(resnet_152.tar.gz)을 `TargetModel`로 지정하고 동일한 엔드포인트를 사용하여 추론을 수행할 수 있습니다.

In [ ]:
%%time

response = runtime_sm_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/x-image",
    TargetModel="resnet_152.tar.gz",
    Body=payload,
)

print(*json.loads(response["Body"].read()), sep="\n")

### endpoint 에 모델 추가

endpoint 를 업데이트하지 않고도 endpoint 에 더 많은 모델을 추가할 수 있습니다. 

아래에 세 번째 모델인 `squeezenet_v1.0`을 추가합니다. 

endpoint 에서 여러 모델을 호스팅하는 것을 보여주기 위해 이 모델은 S3에서 약간 다른 이름으로 10번 복제되었습니다. 

보다 현실적인 시나리오에서는 10개의 새로운 모델이 될 수 있습니다.

In [ ]:
mx.test_utils.download(
    model_path + "squeezenet/squeezenet_v1.0-0000.params", None, "data/squeezenet_v1.0"
)
mx.test_utils.download(
    model_path + "squeezenet/squeezenet_v1.0-symbol.json", None, "data/squeezenet_v1.0"
)
mx.test_utils.download(model_path + "synset.txt", None, "data/squeezenet_v1.0")

with open("data/squeezenet_v1.0/squeezenet_v1.0-shapes.json", "w") as file:
    file.write('[{"shape": [1, 3, 224, 224], "name": "data"}]')

with tarfile.open("data/squeezenet_v1.0.tar.gz", "w:gz") as tar:
    tar.add("data/squeezenet_v1.0", arcname=".")

In [ ]:
file = "data/squeezenet_v1.0.tar.gz"

for x in range(0, 10):
    s3_file_name = "demo-subfolder/squeezenet_v1.0_{}.tar.gz".format(x)
    key = os.path.join(prefix, s3_file_name)
    with open(file, "rb") as file_obj:
        s3.Bucket(bucket).Object(key).upload_fileobj(file_obj)
    models.add(s3_file_name)

print("Number of models: {}".format(len(models)))
print("Models: {}".format(models))


SqueezeNet 모델을 S3에 업로드한 후 endpoint 를 100번 호출하고 각 호출에 대해 S3 prefix 뒤에 있는 12개 모델 중 하나를 무작위로 선택하고 각 호출 응답에서 확률이 가장 높은 레이블 수를 유지합니다.

In [ ]:
%%time

import random
from collections import defaultdict

results = defaultdict(int)

for x in range(0, 100):
    target_model = random.choice(tuple(models))
    response = runtime_sm_client.invoke_endpoint(
        EndpointName=endpoint_name,
        ContentType="application/x-image",
        TargetModel=target_model,
        Body=payload,
    )

    results[json.loads(response["Body"].read())[0]] += 1

print(*results.items(), sep="\n")

### model 갱신

모델을 업데이트하려면 위와 동일한 접근 방식을 따르고 새 모델로 추가합니다. 

예를 들어, `resnet_18.tar.gz` 모델을 다시 training 하고 이 모델을 호출하기 시작하려는 경우 `resnet_18_v2.tar.gz`와 같은 새 이름으로 S3 접두사 뒤에 업데이트된 모델 아티팩트를 업로드한 다음 

`resnet_18.tar.gz` 대신 `resnet_18_v2.tar.gz`를 호출하도록 `TargetModel` 필드를 변경합니다. 

이전 버전의 모델이 여전히 컨테이너 또는 엔드포인트 인스턴스의 스토리지 볼륨에 로드될 수 있기 때문에 Amazon S3의 모델 아티팩트를 덮어쓰지 않도록 구성합니다. 

## (선택) 호스팅 자원 삭제

In [ ]:
sm_client.delete_endpoint(EndpointName=endpoint_name)
sm_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
sm_client.delete_model(ModelName=model_name)